In [8]:
# Author : Dony Varghese
# Date : 05-09-2022

# Synopsis :  Parsing and drawing a visualization of the data containing of Meterological Aerodrome Reports.


#Importing Libraries
import requests as rq

import tkinter as tk

import pandas as pd

import math

import numpy as np



#Exception
class Error(Exception):

    pass


class DataError(Error):

    pass


def run(metars):
    


#Initial data frame
    df = pd.DataFrame(pd.np.nan, index=range(len(metars)), columns=['airport', 'date', 'utc', 'wind_dir','wind_speed', 
                                                        'wind_gust','vis', 'degrees', 'dewpoint', 'altimeter'])

    index = 0

    for entry in metars:
        airport = entry[0:4] #Taking airport name
        df.loc[index,'airport'] = airport
        entry = entry[5:]
        string = entry.split(" ") #Splitting rest of them
        try:#Exception
            
            for word in string: 
                if "RMK" == word: #If remarks come, stop there
                    break 
                
                elif "Z" in word: #Time
                    df.at[index,'date'] = int(word[0:2]) 
                    utc = int(word[2:6]) - 400 
                    if utc < 0: 
                        utc = utc + 2400 
                    if utc <= 1200: 
                        df.loc[index,'utc'] = str(utc) + " AM"
                    else: 
                        utc_1 = utc - 1200 
                        df.loc[index,'utc'] = str(utc_1) + " PM"
                
                elif "KT" in word: #Wind Measures
                    if word[3:5] != "00":
                        if word[0:3] == "VRB":
                            df.loc[index,'wind_dir'] = "Variable"
                            df.at[index,'wind_speed'] = round(int(word[3:5])*1.15, 0)

                        else:    
                            df.at[index,'wind_dir'] = round(int(word[0:3])*1.15,0) 
                            df.at[index,'wind_speed'] = round(int(word[3:5]) *1.15,0)
                    elif word[3:5] == "00": 
                        df.loc[index,'wind_dir'] = 'CALM' 
                        df.loc[index,'wind_speed'] = '' 
                    
                    if word[5] == "G": 
                        df.loc[index,'wind_gust'] = word[6:8]
                    else: 
                        df.loc[index,'wind_gust'] = '0'
                
                elif "SM" in word: #Visibility
                    if word[1] =="/":
                        df.loc[index,'vis'] = word[0:3]
                    elif (word[0] == "P" or word[0] == "M"):
                        df.loc[index,'vis'] = word[1]
                    else:
                        df.loc[index,'vis'] = word[0:2]
               
                #for temperature
                elif word[2] == "/" and len(word) == 5:
                    C = int(word[0:2])
                    F = 32 + C*(9/5)
                    df.loc[index,'degrees'] = round(F,2)
                    DC = int(word[4:]) 
                    DF = 32 + DC*(9/5)
                    df.loc[index,'dewpoint'] = round(DF,2)

                elif "/" in word and len(word) == 7:
                    C = int(word[1:3])
                    F = 32 - C*(9/5)
                    df.loc[index,'degrees'] = round(F,2)
                    DC = int(word[5:]) 
                    DF = 32 - DC*(9/5)
                    df.loc[index,'dewpoint'] = round(DF,2)

                elif word[2] == "/" and len(word) == 6:
                    C = int(word[0:2])
                    F = 32 + C*(9/5)
                    df.loc[index,'degrees'] = round(F,2)
                    DC = int(word[4:]) 
                    DF = 32 - DC*(9/5)
                    df.loc[index,'dewpoint'] = round(DF,2)

                elif word[0] =="A" and len(word) ==5: #Altimeter
                    df.loc[index,'altimeter'] = word[1:3]+'.'+word[3:5]
                    
                    
                    
        except DataError:#Exception
            print("Data Error!")
            print()
        index = index + 1




    # Create the root Tk()
    root = tk.Tk()
    # Set the title
    root.title("COSC505 - Weather")
    # Create two frames, the list is on top of the Canvas
    list_frame = tk.Frame(root)
    draw_frame = tk.Frame(root)
    # Set the list grid in c,r = 0,0
    list_frame.grid(column=0, row=0)
    # Set the draw grid in c,r = 0,1
    draw_frame.grid(column=0,row=1)

    # Create the canvas on the draw frame, set the width to 800 and height to 600
    canvas = tk.Canvas(draw_frame, width=800, height=600)
    # Reset the size of the grid
    canvas.pack()
    # THESE ARE EXAMPLES! You need to populate this list with the available airports in the METAR 
    # which is given by metar parameter passed into this function.
    choices = df['airport'].tolist()
    

    # Create a variable that will store the currently selected choice.
    listvar = tk.StringVar(root)
    # Immediately set the choice to the first element. Double check to make sure choices[0] is valid!
    listvar.set(choices[0])

    # Create the dropdown menu with the given choices and the update variable. This is stored on the
    # list frame. You must make sure that choices is already fully populated.
    dropdown = tk.OptionMenu(list_frame, listvar, *choices)
    # The dropdown menu is on the top of the screen. This will make sure it is in the middle.
    dropdown.grid(row=0,column=1)
    # This function is called whenever the user selects another. Change this as you see fit.
    def drop_changed(*args):
            canvas.delete("airport_text")
            canvas.delete("time")
            canvas.delete("tag1")
            canvas.delete("tag2")
            canvas.delete("tag3")
            canvas.delete("tag4")
            canvas.delete("tag5")
            canvas.delete("tag6")
            canvas.delete("tag7")
            canvas.delete("tag8")
            canvas.delete("tag9")
            
            canvas.create_text(100, 100, text=listvar.get(), fill="red", tags="airport_text",font=("Arial", 35))
            index_1 = df[df['airport'] == listvar.get()].index.values
            
            index_1 = int(index_1)
            
            canvas.create_oval(300, 100, 400, 200, fill ="grey")
            
 
            if type(df.loc[index_1,'wind_dir'])!= str:
                canvas.create_line(350, 150, 350+50*np.cos(math.radians(90-int(df.loc[index_1,'wind_dir']))), 
                                                                           150+50*np.sin(math.radians(90-int(df.loc[index_1,'wind_dir']))),
                                                                                        fill ="white",width=4)
                canvas.create_text(320,220, text = str (df.at[index_1,'wind_speed']) + "MPH", tags="tag1")

            else: 
                canvas.create_text(220,220, text = "CALM", tags="tag1")

            canvas.create_oval(345, 145, 355, 155, fill ="red")
            
            canvas.create_text(100,140, text = df.at[index_1,'utc'], fill = "blue",tags="time", font=("Arial", 35))

            
            canvas.create_text(320,235, text = "Gust:" + str (df.at[index_1,'wind_gust']) + "MPH",tags="tag2")
            canvas.create_oval(300, 275, 400, 375, fill = "black")
            canvas.create_text(350, 325, text = df.at[index_1,'altimeter'], fill = "white",tags="tag3")
            canvas.create_rectangle(500, 100, 600, 300, width = 4)
            canvas.create_rectangle(500, 300 - (df.at[index_1,'degrees']/100)*200, 600, 300, fill = 'red',width = 4,tags="tag4")
            canvas.create_rectangle(500, 300 - (df.at[index_1,'dewpoint']/100)*200, 600, 300, fill = 'blue', width = 4, tags="tag5")
            canvas.create_text(550, 325, text = str(df.at[index_1,'degrees']) +"F", fill = "red", tags="tag6")
            canvas.create_text(550, 340, text = str(df.at[index_1,'dewpoint'])+"F", fill = "blue", tags="tag7")
            canvas.create_rectangle(300, 400, 700, 500, width = 4)
            canvas.create_rectangle(300, 400, 300 + int(df.at[index_1,'vis'])*40, 500, fill = "orange", width = 4,tags="tag8")
            canvas.create_text(320, 515, text = str(df.at[index_1,'vis']) +" SM", fill = "green",tags="tag9")
        
    # Listen for the dropdown to change. When it does, the function drop_changed is called.
    listvar.trace('w', drop_changed)
    # You need to draw the text manually with the first choice.
    drop_changed()
    # mainloop() is necessary for handling events
    tk.mainloop()

# Entry point for running programs
if __name__ == "__main__":
    
    lnk = "https://aviationweather.gov/metar/data?ids=KTYS%2CKBNA%2CKLAS%2CKLSV%2CKVGT&format=raw&date=&hours=0" 
    data = rq.get(lnk) 

    needle = "<!-- Data starts here -->" 
    needle_position = data.text.find(needle) + len(needle) 
    data = data.text[needle_position:] 
    needle = "<!-- Data ends here -->" 
    needle_position = data.find(needle) 
    data = data[:needle_position]

    metars = []
    needle = "<code>"
    while True: 
        pos_s = data.find(needle) 
        if pos_s == -1: 
            break 
        pos_e = data.find("</code>") 
        if pos_e != -1: 
            apt = data[pos_s + len(needle):pos_e] 
            data = data[pos_e + len(needle) + 1:] 
        else: 
            apt = data[pos_s + len(needle):] 
            data = ""
        metars.append(apt) 
    data = metars
    run(data)
    